In [1]:
# Use with NHL_Model_Data_Transform_v5.py
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt

from tensorflow import keras


rng = 69

In [2]:
#df = pd.read_csv(r"C:\Users\zchodaniecky\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Transformed.csv")
df = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_Transformed.csv")

In [3]:
df = df.drop(columns=['team_Home','team_Away','gameId','home_or_away_Home','home_or_away_Away','win_or_lose_Away'])

In [4]:
# df.hist(bins=50, figsize=(20,15))
# plt.show()

In [5]:
# Heatmap to visualize correlations
corr_matrix = df.corr(numeric_only=True)
# sns.heatmap(corr_matrix, cmap="PiYG")

In [6]:
corr_matrix['win_or_lose_Home'].sort_values(ascending=False)

win_or_lose_Home             1.000000
seasonPointsPerGame_Home     0.301313
fenwickPercentageAvg_Home    0.159233
goalDiffAvg_Home             0.111563
pointsFromGameAvg_Home       0.105887
hitsDiffAvg_Away             0.074182
reboundsForAvg_Home          0.057896
penaltiesAgainstTotal        0.028917
penaltiesForTotal           -0.019003
hitsDiffAvg_Home            -0.039441
reboundsForAvg_Away         -0.059495
pointsFromGameAvg_Away      -0.113037
goalDiffAvg_Away            -0.114056
fenwickPercentageAvg_Away   -0.166292
seasonPointsPerGame_Away    -0.293767
Name: win_or_lose_Home, dtype: float64

In [7]:
# Visualize distributions of attributes
# from pandas.plotting import scatter_matrix

# attributes = ['fenwickPercentageAvg_Home','goalDiffAvg_Home','seasonPointsPerGame_Home','seasonPointsPerGame_Away']
# scatter_matrix(df[attributes], figsize=(12,8));

In [8]:
# These are tentative just to test
df_Final = df
# df_Final = df.drop(columns=['corsiPercentageAvg_Away','penaltiesAgainstTotal','shotsOnGoalDiffAvg_Away'])

In [9]:
from sklearn.model_selection import train_test_split

# Split into 80% training data and 20% testing data
train_set, test_set = train_test_split(df_Final, test_size=0.2, random_state=rng)

nhl = train_set.drop('win_or_lose_Home', axis=1)
nhl_labels = train_set['win_or_lose_Home'].copy()

X_train = train_set.drop('win_or_lose_Home', axis=1)
Y_train = train_set['win_or_lose_Home'].copy()

X_valid = train_set.drop('win_or_lose_Home', axis=1)
Y_valid = train_set['win_or_lose_Home'].copy()

X_test = test_set.drop('win_or_lose_Home', axis=1)
Y_test = test_set['win_or_lose_Home'].copy()

df_Final = df_Final.drop('win_or_lose_Home', axis=1)

In [10]:
# Pipeline constructor used to run transformation steps in order
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

X_train_prepared= num_pipeline.fit_transform(X_train)
X_valid_prepared= num_pipeline.fit_transform(X_valid)
X_test_prepared= num_pipeline.fit_transform(X_test)


In [11]:
# Fine Tune the hyperparameters of RandomForest model
# from sklearn.model_selection import GridSearchCV

# param_grid = [{'n_estimators' : [100,500,1000], 'max_leaf_nodes': [10,16,25,50], 'n_jobs' : [-1]},
#               {'bootstrap': [False], 'n_estimators' : [100,500,1000], 'max_leaf_nodes': [10,16,25,50], 'n_jobs' : [-1]},]

# rnd_clf_test = RandomForestClassifier()

# grid_search = GridSearchCV(rnd_clf_test, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

# grid_search.fit(nhl_prepared,nhl_labels)

# grid_search.best_params_

In [12]:
# cvres = grid_search.cv_results_
# results = []
# for mean_scores, params in zip(cvres["mean_test_score"], cvres["params"]):
#     results.append([np.sqrt(-mean_scores),params])

# results2 = sorted(results, key=lambda x: x[0])
# print(*results2, sep = '\n')

In [13]:
# # baseline model
# from tensorflow import keras


# # create model
# base_model = keras.models.Sequential([
#     keras.Input(shape=(X_train.shape[1],)),
#     keras.layers.Dense(30, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')
#                        ])
    
# # Compile model
# base_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [14]:
def build_model(n_hidden=1, n_neurons=50, learning_rate=1e-3, input_shape=[X_train.shape[1]]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons,activation='selu',kernel_initializer="lecun_normal"))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy',optimizer=optimizer)
    return model

In [15]:
from scikeras.wrappers import KerasClassifier

# Seems like you need to specify some items again when using the scikeras wrapper
keras_clf = KerasClassifier(model=build_model, n_hidden=1, n_neurons=30, learning_rate=3e-3)


In [16]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    'model__n_hidden': [0,1,2,3],
    'model__n_neurons': np.arange(1,50),
    'model__learning_rate' : reciprocal(1e-5, 1e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_clf, param_distribs, n_iter=10, cv=3, error_score='raise')

early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
rnd_search_cv.fit(X_train_prepared,Y_train, epochs=100,
                 validation_data = (X_valid_prepared, Y_valid),
                 callbacks=[early_stopping_cb])

Epoch 1/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.7970 - val_loss: 0.7845
Epoch 2/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7839 - val_loss: 0.7765
Epoch 3/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7754 - val_loss: 0.7692
Epoch 4/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7868 - val_loss: 0.7623
Epoch 5/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7551 - val_loss: 0.7560
Epoch 6/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7645 - val_loss: 0.7501
Epoch 7/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7463 - val_loss: 0.7445
Epoch 8/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7616 - val_loss: 0.7393
Epoch 9/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7419 - val_loss: 0.7344
Epoch 10/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7547 - val_loss: 0.7297
Epoch 11/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7292 - val_loss: 0.7253
Epoch 12/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7

Epoch 37/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6330 - val_loss: 0.6325
Epoch 38/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6395 - val_loss: 0.6310
Epoch 39/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6403 - val_loss: 0.6294
Epoch 40/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6338 - val_loss: 0.6280
Epoch 41/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6532 - val_loss: 0.6265
Epoch 42/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6204 - val_loss: 0.6252
Epoch 43/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6250 - val_loss: 0.6239
Epoch 44/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6344 - val_loss: 0.6226
Epoch 45/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6330 - val_loss: 0.6215
Epoch 46/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6403 - val_loss: 0.6203
Epoch 47/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6087 - val_loss: 0.6192
Epoch 48/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - 

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6076 - val_loss: 0.6050
Epoch 73/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5953 - val_loss: 0.6046
Epoch 74/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5924 - val_loss: 0.6043
Epoch 75/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6094 - val_loss: 0.6039
Epoch 76/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5951 - val_loss: 0.6035
Epoch 77/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6031 - val_loss: 0.6032
Epoch 78/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5929 - val_loss: 0.6029
Epoch 79/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6004 - val_loss: 0.6026
Epoch 80/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5950 - val_loss: 0.6023
Epoch 81/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5980 - val_loss: 0.6021
Epoch 82/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6005 - val_loss: 0.6018
Epoch 83/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5907 

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6066 - val_loss: 0.5953
Epoch 7/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5983 - val_loss: 0.5951
Epoch 8/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5627 - val_loss: 0.5928
Epoch 9/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5918 - val_loss: 0.5901
Epoch 10/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5736 - val_loss: 0.5932
Epoch 11/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5795 - val_loss: 0.5901
Epoch 12/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5975 - val_loss: 0.5891
Epoch 13/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5746 - val_loss: 0.5889
Epoch 14/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5953 - val_loss: 0.5877
Epoch 15/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6032 - val_loss: 0.5881
Epoch 16/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5826 - val_loss: 0.5869
Epoch 17/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5854 - v

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5743 - val_loss: 0.5789
Epoch 42/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5729 - val_loss: 0.5788
Epoch 43/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5612 - val_loss: 0.5788
Epoch 44/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5508 - val_loss: 0.5775
Epoch 45/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5561 - val_loss: 0.5767
Epoch 46/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5621 - val_loss: 0.5781
Epoch 47/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5587 - val_loss: 0.5777
Epoch 48/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5507 - val_loss: 0.5783
Epoch 49/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5472 - val_loss: 0.5785
Epoch 50/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5707 - val_loss: 0.5815
Epoch 51/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5448 - val_loss: 0.5795
Epoch 52/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5491 

...

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5754 - val_loss: 0.5853
Epoch 33/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5707 - val_loss: 0.5863
Epoch 34/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5753 - val_loss: 0.5848
Epoch 35/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5855 - val_loss: 0.5854
Epoch 36/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5755 - val_loss: 0.5874
Epoch 37/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5715 - val_loss: 0.5858
Epoch 38/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.5784 - val_loss: 0.5851
Epoch 39/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5614 - val_loss: 0.5845
Epoch 40/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5708 - val_loss: 0.5857
Epoch 41/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5790 - val_loss: 0.5852
Epoch 42/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5609 - val_loss: 0.5844
Epoch 43/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5672 

Epoch 68/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5823 - val_loss: 0.5864
Epoch 69/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5727 - val_loss: 0.5862
Epoch 70/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5812 - val_loss: 0.5861
Epoch 71/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5777 - val_loss: 0.5859
Epoch 72/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5856 - val_loss: 0.5857
Epoch 73/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5847 - val_loss: 0.5854
Epoch 74/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5714 - val_loss: 0.5853
Epoch 75/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5880 - val_loss: 0.5853
Epoch 76/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5750 - val_loss: 0.5850
Epoch 77/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5780 - val_loss: 0.5849
Epoch 78/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5703 - val_loss: 0.5847
Epoch 79/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - 

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6872 - val_loss: 0.6639
Epoch 3/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6440 - val_loss: 0.6433
Epoch 4/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6210 - val_loss: 0.6306
Epoch 5/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6164 - val_loss: 0.6223
Epoch 6/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6073 - val_loss: 0.6166
Epoch 7/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6015 - val_loss: 0.6118
Epoch 8/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6089 - val_loss: 0.6083
Epoch 9/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5977 - val_loss: 0.6056
Epoch 10/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5936 - val_loss: 0.6034
Epoch 11/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5857 - val_loss: 0.6015
Epoch 12/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6028 - val_loss: 0.6000
Epoch 13/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5871 - val_l

102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5971 - val_loss: 0.5964
Epoch 38/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5914 - val_loss: 0.5960
Epoch 39/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6004 - val_loss: 0.5956
Epoch 40/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5959 - val_loss: 0.5952
Epoch 41/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5816 - val_loss: 0.5949
Epoch 42/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5898 - val_loss: 0.5946
Epoch 43/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5850 - val_loss: 0.5943
Epoch 44/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5924 - val_loss: 0.5941
Epoch 45/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5956 - val_loss: 0.5939
Epoch 46/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5920 - val_loss: 0.5937
Epoch 47/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6004 - val_loss: 0.5935
Epoch 48/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 

RandomizedSearchCV(cv=3, error_score='raise',
                   estimator=KerasClassifier(learning_rate=0.003, model=<function build_model at 0x000001C0900D9A80>, n_hidden=1, n_neurons=30),
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001C0901192D0>,
                                        'model__n_hidden': [0, 1, 2, 3],
                                        'model__n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])})

In [17]:
#print(rnd_search_cv.best_params_)
#print(rnd_search_cv.best_score_)

In [18]:
print(rnd_search_cv.best_params_)
print(rnd_search_cv.best_score_)

{'model__learning_rate': 0.00039447935272202665, 'model__n_hidden': 0, 'model__n_neurons': 35}
0.6778646799619029


In [19]:
model = rnd_search_cv.best_estimator_.model

In [20]:
from scikeras.wrappers import KerasClassifier

# Seems like you need to specify some items again when using the scikeras wrapper
keras_clf = KerasClassifier(model=model, epochs=100, batch_size=5, verbose=0, random_state = 42,
                           loss='binary_crossentropy',optimizer='adam',metrics='accuracy')

# Stop early if model is not getting better after # of patience epochs, restore to best model
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
#checkpoint_cb = keras.callbacks.ModelCheckpoint("Regression_Sequential.keras", save_best_only = True)

keras_clf.fit(X_train_prepared, Y_train, validation_data = (X_valid_prepared, Y_valid),
             callbacks=[early_stopping_cb])



KerasClassifier(
	model=<function build_model at 0x000001C0900D9A80>
	build_fn=None
	warm_start=False
	random_state=42
	optimizer=adam
	loss=binary_crossentropy
	metrics=accuracy
	batch_size=5
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=100
	class_weight=None
)

In [21]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

#kfold = StratifiedKFold(n_splits=5, shuffle=True)
#results = cross_val_score(keras_clf, X_train_prepared, Y_train, cv=kfold)

#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [22]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
Y_pred = cross_val_predict(keras_clf, X_train_prepared, Y_train, cv=kfold)

from sklearn.metrics import confusion_matrix
confusion_matrix(Y_train, Y_pred)
# [Predict Loss - Actual Loss  |  Predict Win - Actual Loss]
# [Predict Loss - Actual Win   |  Predict Win - Actual Win]

array([[ 908,  607],
       [ 497, 1232]], dtype=int64)

In [23]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

print('When claiming is a win, correct % is ', precision_score(Y_train, Y_pred))
print('Detected % of wins' ,recall_score(Y_train, Y_pred))
print('F1 score is ', f1_score(Y_train, Y_pred))
print('Training Accuracy score is ', accuracy_score(Y_train, Y_pred))


When claiming is a win, correct % is  0.6699293094072866
Detected % of wins 0.7125506072874493
F1 score is  0.6905829596412556
Training Accuracy score is  0.6596794081381011


In [24]:
# Train Accuracy
Y_train_pred = keras_clf.predict(X_train_prepared)
print(accuracy_score(Y_train, Y_train_pred))

Y_test_pred = keras_clf.predict(X_test_prepared)
print(accuracy_score(Y_test, Y_test_pred))

0.717940813810111
0.6588669950738916


In [25]:
#df_Predict = pd.read_csv(r"C:\Users\zchodaniecky\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Predict.csv")
df_Predict = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_Predict.csv")

In [26]:
X_real_predict = df_Predict.iloc[:,2:]

np.array((df_Predict.iloc[:,1].values,keras_clf.predict(X_real_predict))).T

ValueError: Found array with 0 sample(s) (shape=(0, 14)) while a minimum of 1 is required.